In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import prover_gen_settings, verifier_setup, prover_gen_proof, verifier_verify

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
# srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

## Step 2
- User defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `create_model(computation)` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [6]:
from zkstats.computation import State, create_model


def computation(state: State, x: list[torch.Tensor]):
    x_0 = x[0]
    out_0 = state.median(x_0)
    out_1 = state.median(x_0)
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

_, prover_model = create_model(computation)
prover_gen_settings([data_path], comb_data_path, prover_model, prover_model_path, "default", "resources", settings_path)


/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_56057/1617438771.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_56057/1617438771.py:8: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/computation.py:75: TracerWarning: torch.tensor results are registered as constants in

==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-34048,33628],"logrows":17,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":1312,"total_assignments":289,"total_const_size":52,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[1312,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [7]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(prover_model_path, prover_compiled_model_path, settings_path, vk_path, pk_path )

print("=======================================")
# Prover generates proof
# print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 23.758974313735962 seconds
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 59.6015625
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[1969817931821079592, 14093252634055327186, 3010599488369023793, 2676483216109330922], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10077958863098660019, 689600963559187911, 14561112591056096131, 381975124955332008]]], 'proof': '0d74573c6f907eddd298a72bc0d97b89e8b39d3fbb63b132fb6e9a72e566f5b41dcee3495d40f3c0ce73f2469c5e3d7b48d69b25a37ce19d15536c2932d72e9c0838b9435df676b85297131d7b3e791ee365137a82cf577503ffb33102d520c4035446c0cf0045c3843f163586289422aa07d07ea9be17c207f54110fe7a3d00047e0c4de126417a3f04612935c7f516afb647eb5cc7cddca9121d487e2483651f84e3676caf3cd1646cea8f8e2451226dae62df907ed6a2e68f38d005cf08562cf756873193135047aa8a8d49f58672b4d7794c97aaf8f4af9497bb67cc12750c7abb9cd75076e03b8888d8975c8764311a62ac32545333ccef4785e130fda7160ff719ebb890f2081264d4bba44927d8be5919e1c666674c2755170ba123220f7dd8ce25687e6ac5d95e241feaab2712c94983f67c29a772c8d970c2bd7169224746428dec36a5f3c16020a8b6ee18259514335861468bf6e84d473d9221fb

In [8]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[1969817931821079592, 14093252634055327186, 3010599488369023793, 2676483216109330922], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10077958863098660019, 689600963559187911, 14561112591056096131, 381975124955332008]]]
proof boolean:  1.0
proof result 1 : 59.6015625
verified
